Analysis of the TEMPO Polyco.C code

We try to figure out the following things:
1. How does polyco decide the MJD values for phase prediction and how does it depend on the input paramters ?
2. To crosscheck the LST and HA values calculated by tempo with astropy 
3. Can we adjust the imput paramters such that we can get polyco to give phase at a particular value?

In [1]:
#Basic MJD and Date Code blocks 
from astropy.coordinates import EarthLocation, AltAz, SkyCoord
from astropy.time import Time
import astropy.units as u
observation_date = '2023-12-25T08:36:09'
observation_time = Time(observation_date,scale='utc')
observation_time.mjd #get obstime in MJD 

60303.3584375

In [2]:
#This is just the opposite (lazy)
mjd = 60298.66666666666
time_object = Time(mjd, format='mjd')
datetime_object = time_object.datetime
datetime_object #Date and time : UTC 

datetime.datetime(2023, 12, 20, 15, 59, 59, 999999)

The Following code is a python alternative of the polyco function that calculates the TMID_start value. This was done to analyse the parameters that are actually required to decide the TMID_start value and I could explore what other paramters was tempo calculating and how it was arriving to certain results. 

In [9]:
#Working Code that just recalculates mjd_start value from polyco 

import datetime
import math

# Given values
#transit_lst_str = "08:36:09" #LST transit at afmjd
ra_str = "08h35m20.5554429s" #RA of Pulsar 
afmjd =  60298#60268 #60297  # Example value for afmjd(MJD_start)
maxha = 6 #  maxha 
tspan = 60  #  tspan (min)

# Observatory coordinates  1656342.30    5797947.77 
obs_x =  1656342.30  # Replace with the actual x-coordinate
obs_y = 5797947.77 # Replace with the actual y-coordinate

# Convert transit LST to datetime
#transit_lst_time = datetime.datetime.strptime(transit_lst_str, "%H:%M:%S").time()

# Convert RA to hours
ra_hours = (8 + (35/60) + (20.5554429/3600))

# Conversion of UT into local sidereal time
solsid = 1.002737909

#  LST transit in hours (taken from the NCRA transit time calculator)
#hlst_gmrt = transit_lst_time.hour + transit_lst_time.minute/60 + transit_lst_time.second/3600 

# Calculate observatory longitude
alng = math.atan2(-obs_y, obs_x)

# Conversion of UT into local sidereal time
solsid = 1.002737909
hlst = 24.0 * ((solsid * afmjd + 0.154374 - alng / (2 * math.pi)) % 1.0)

 
rax = int(ra_hours) + int((ra_hours - int(ra_hours)) * 60.0 + 2) / 60.0 #RA in hours at epoch

wait_gmrt = (rax - hlst ) / solsid #GMRT # Calculate wait : Local hour angle

# Add a day if wait is not in range #This is polyco criterion which takes care of rise and setting of source
if wait_gmrt < -maxha:
    wait_gmrt += 24.0/solsid

# Calculate MJD1
mjd1_gmrt = afmjd + (wait_gmrt - (maxha)) / 24.0 + (tspan / (2.0 * 1440.0)) 
#mjd1_gmrt_new = afmjd + ((rax - 24.0 * ((solsid * afmjd + 0.154374 - alng / (2 * math.pi)) % 1.0)) - maxha) / 24.0 + tspan / (2.0 * 1440.0)

mjd1_gmrt_frac = round(48.0 * (mjd1_gmrt - int(mjd1_gmrt))) / 48.0
mjd1_gmrt_frac_new = round(48.0 * (mjd1_gmrt - int(mjd1_gmrt))) / 48.0
ntmp1 = int(mjd1_gmrt_frac * 1.0e8)
ntmp2 = int((mjd1_gmrt_frac * 1.0e8 - ntmp1) * 100.0)
mjd1 = ntmp2 * 1e-10 + ntmp1 * 1.0e-8
mjd1_gmrt_frac=mjd1
mjd1_gmrt_round = int(mjd1_gmrt) + mjd1_gmrt_frac#round(48.0 * (mjd1_gmrt - int(mjd1_gmrt))) / 48.0
#mjd1_gmrt_round_new = int(mjd1_gmrt_new) + mjd1_gmrt_frac_new
print(f"MJD_gmrt_round: {mjd1_gmrt_round}")

MJD_gmrt_round: 60297.6875


Test 1 : Validity and Substitution of LST 

From the above code we can see that lst is calulated in the parameter hlst which is further used in the calculation local hour hangle and finally to get the tmid_mjd_start. 

In [6]:
#This is to check the LST at GMRT at some afmjd #This code block is used to compare the hlst values with the lst from astropy 


#afmjd

afmjd = 60298

# Given geocentric coordinates GMRT from observatory.dat in tempo2 
obs_x = 1656342.30 * u.m  # x-coordinate in meters
obs_y = 5797947.77 * u.m  # y-coordinate in meters
obs_z = 2073243.16 * u.m  # z-coordinate in meters


# Create EarthLocation object from geocentric coordinates
obs_location = EarthLocation.from_geocentric(x=obs_x, y=obs_y, z=obs_z)

# Create Astropy Time object (assuming afmjd is the Modified Julian Date at the start of observations)
time_utc = Time(afmjd, format='mjd', scale='utc')

# Calculate LST using Astropy
lst = time_utc.sidereal_time('mean', longitude=obs_location.lon)

# Get geodetic coordinates
latitude = obs_location.lat.deg
longitude = obs_location.lon.deg
elevation = obs_location.height.value

# Calculate Hour Angle at transit
ha_at_transit = lst.hour - ra_hours

# Normalize Hour Angle
transit_lst = lst.hour - ha_at_transit
#transit_lst = transit_lst.wrap_at(24 * u.hourangle)  # Normalize to [0, 24) hours


# Print the results
print(f"Latitude: {latitude:.6f} degrees")
print(f"Longitude: {longitude:.6f} degrees")
print(f"Elevation: {elevation:.2f} meters")
print(f"Local Sidereal Time: {lst:.6f} hours")
print(f"hlst given by polyco: {hlst} hrs")
#print(f"hlst_gmrt from GMRT transit:{hlst_gmrt} hrs")
print(f"Transit LST from astropy: {transit_lst}")

Latitude: 19.093003 degrees
Longitude: 74.056561 degrees
Elevation: 497.00 meters
Local Sidereal Time: 10.825430 hourangle hours
hlst given by polyco: 10.812565245141741 hrs
Transit LST from astropy: 8.589043178583333


In [142]:
# LST sustitution  test 

import datetime
import math

# Given values
#transit_lst_str = "08:36:09" #LST transit at afmjd
ra_str = "08h35m20.5554429s" #RA of Pulsar 
afmjd =  60298#60268 #60297  # Example value for afmjd(MJD_start)
maxha = 6 #  maxha 
tspan = 60  #  tspan (min)

# Observatory coordinates  1656342.30    5797947.77 
obs_x =  1656342.30  # Replace with the actual x-coordinate
obs_y = 5797947.77 # Replace with the actual y-coordinate

# Convert transit LST to datetime
#transit_lst_time = datetime.datetime.strptime(transit_lst_str, "%H:%M:%S").time()

# Convert RA to hours
ra_hours = (8 + (35/60) + (20.5554429/3600))

# Conversion of UT into local sidereal time
solsid = 1.002737909

#  LST transit in hours (taken from the NCRA transit time calculator)
#hlst_gmrt = transit_lst_time.hour + transit_lst_time.minute/60 + transit_lst_time.second/3600 

# Calculate observatory longitude
alng = math.atan2(-obs_y, obs_x)

# Conversion of UT into local sidereal time
solsid = 1.002737909
hlst = 10.825430  #Substituting the lst calculated using astropy           #24.0 * ((solsid * afmjd + 0.154374 - alng / (2 * math.pi)) % 1.0)

 
rax = int(ra_hours) + int((ra_hours - int(ra_hours)) * 60.0 + 2) / 60.0 #RA in hours at epoch

wait_gmrt = (rax - hlst ) / solsid #GMRT # Calculate wait : Local hour angle

# Add a day if wait is not in range #This is polyco criterion which takes care of rise and setting of source
if wait_gmrt < -maxha:
    wait_gmrt += 24.0/solsid

# Calculate MJD1
mjd1_gmrt = afmjd + (wait_gmrt - (maxha)) / 24.0 + (tspan / (2.0 * 1440.0)) 
#mjd1_gmrt_new = afmjd + ((rax - 24.0 * ((solsid * afmjd + 0.154374 - alng / (2 * math.pi)) % 1.0)) - maxha) / 24.0 + tspan / (2.0 * 1440.0)

mjd1_gmrt_frac = round(48.0 * (mjd1_gmrt - int(mjd1_gmrt))) / 48.0
mjd1_gmrt_frac_new = round(48.0 * (mjd1_gmrt - int(mjd1_gmrt))) / 48.0
ntmp1 = int(mjd1_gmrt_frac * 1.0e8)
ntmp2 = int((mjd1_gmrt_frac * 1.0e8 - ntmp1) * 100.0)
mjd1 = ntmp2 * 1e-10 + ntmp1 * 1.0e-8
mjd1_gmrt_frac=mjd1
mjd1_gmrt_round_astropy_lst = int(mjd1_gmrt) + mjd1_gmrt_frac#round(48.0 * (mjd1_gmrt - int(mjd1_gmrt))) / 48.0
#mjd1_gmrt_round_new = int(mjd1_gmrt_new) + mjd1_gmrt_frac_new
print(f"MJD_gmrt_round_astropy_lst: {mjd1_gmrt_round_astropy_lst} , MJD_1_gmrt_polyco_lst: {mjd1_gmrt_round}")

MJD_gmrt_round_astropy_lst: 60297.6875 , MJD_1_gmrt_polyco_lst: 60297.6875


Test 2 : Transit LST  
It is clear from above that we cannot get afmjd by solving the equation to get mjd_grmt_1 as it is non-linear . However we may try to see get the afmjd at the transit where the HA=0 and the LST = RA so majority of the parameters are trivial

at TRANSIT :
wait = 0 #Local hour angle 
hlst = rax

mjd1_gmrt = afmjd + (wait_gmrt - (maxha)) / 24.0 + (tspan / (2.0 * 1440.0)) 
mjd1_gmrt = afmjd + (maxha) / 24.0 + (tspan / (2.0 * 1440.0)) 
afmjd = mjd1_gmrt - (maxha) / 24.0 + (tspan / (2.0 * 1440.0)) 

In [141]:
afmjd = 60298


# Given geocentric coordinates GMRT from observatory.dat in tempo2 
obs_x = 1656342.30 * u.m  # x-coordinate in meters
obs_y = 5797947.77 * u.m  # y-coordinate in meters
obs_z = 2073243.16 * u.m  # z-coordinate in meters


# Create EarthLocation object from geocentric coordinates
obs_location = EarthLocation.from_geocentric(x=obs_x, y=obs_y, z=obs_z)

# Astropy Time object
time_utc = Time(afmjd, format='mjd', scale='utc')

# Calculate LST using Astropy
lst = time_utc.sidereal_time('mean', longitude=obs_location.lon)

# Get geodetic coordinates
latitude = obs_location.lat.deg
longitude = obs_location.lon.deg
elevation = obs_location.height.value

# Calculate Hour Angle at transit
ha_at_transit = lst.hour - ra_hours

# Normalize Hour Angle
transit_lst = lst.hour - ha_at_transit
transit_lst = (transit_lst) # Normalize to [0, 24) hours


# Print the results
print(f"Latitude: {latitude:.6f} degrees")
print(f"Longitude: {longitude:.6f} degrees")
print(f"Elevation: {elevation:.2f} meters")
print(f"Local Sidereal Time: {lst:.6f} hours")
print(f"hlst given by polyco: {hlst} hrs")
#print(f"hlst_gmrt from GMRT transit:{hlst_gmrt} hrs")
print(f"Transit LST from astropy: {transit_lst}")

Latitude: 19.093003 degrees
Longitude: 74.056561 degrees
Elevation: 497.00 meters
Local Sidereal Time: 10.825430 hourangle hours
hlst given by polyco: 8.406038785062265 hrs
Transit LST from astropy: 8.589043178583333


In [24]:
from astropy.coordinates import EarthLocation, AltAz, SkyCoord
from astropy.time import Time
import astropy.units as u

#This is just the opposite (lazy)
mjd = 60298
time_object = Time(mjd, format='mjd')
datetime_object = time_object.datetime
print(datetime_object) #Date and time : UTC 


observation_date = '2023-12-20T08:36:09'
observation_time = Time(observation_date,scale='utc')
observation_time.mjd #get obstime in MJD 

2023-12-20 00:00:00


60298.3584375

In [101]:
mjd = 60297.900001
time_object = Time(mjd, format='mjd')
datetime_object = time_object.datetime
print(datetime_object) #Date and time : UTC 

2023-12-19 21:36:00.086400


In [116]:
#Working Code that just recalculates mjd_start value from polyco 

import datetime
import math

# Given values
#transit_lst_str = "08:36:09" #LST transit at afmjd
ra_str = "08h35m20.5554429s" #RA of Pulsar 
afmjd =  60298.08311218355902636#60268 #60297  # Example value for afmjd(MJD_start)
maxha = 6 #  maxha 
tspan = 60  #  tspan (min)

# Observatory coordinates  1656342.30    5797947.77 
obs_x =  1656342.30  # Replace with the actual x-coordinate
obs_y = 5797947.77 # Replace with the actual y-coordinate

# Convert transit LST to datetime
#transit_lst_time = datetime.datetime.strptime(transit_lst_str, "%H:%M:%S").time()

# Convert RA to hours
ra_hours = (8 + (35/60) + (20.5554429/3600))

# Conversion of UT into local sidereal time
solsid = 1.002737909

#  LST transit in hours (taken from the NCRA transit time calculator)
#hlst_gmrt = transit_lst_time.hour + transit_lst_time.minute/60 + transit_lst_time.second/3600 

# Calculate observatory longitude
alng = math.atan2(-obs_y, obs_x)

# Conversion of UT into local sidereal time
solsid = 1.002737909
hlst = 24.0 * ((solsid * afmjd + 0.154374 - alng / (2 * math.pi)) % 1.0)

 
rax = int(ra_hours) + int((ra_hours - int(ra_hours)) * 60.0 + 2) / 60.0 #RA in hours at epoch

wait_gmrt = (rax - hlst ) / solsid #GMRT # Calculate wait : Local hour angle

# Add a day if wait is not in range #This is polyco criterion which takes care of rise and setting of source
if wait_gmrt < -maxha:
    wait_gmrt += 24.0/solsid

# Calculate MJD1
mjd1_gmrt = afmjd + (wait_gmrt - (maxha)) / 24.0 + (tspan / (2.0 * 1440.0)) 
#mjd1_gmrt_new = afmjd + ((rax - 24.0 * ((solsid * afmjd + 0.154374 - alng / (2 * math.pi)) % 1.0)) - maxha) / 24.0 + tspan / (2.0 * 1440.0)

mjd1_gmrt_frac = round(48.0 * (mjd1_gmrt - int(mjd1_gmrt))) / 48.0
mjd1_gmrt_frac_new = round(48.0 * (mjd1_gmrt - int(mjd1_gmrt))) / 48.0
ntmp1 = int(mjd1_gmrt_frac * 1.0e8)
ntmp2 = int((mjd1_gmrt_frac * 1.0e8 - ntmp1) * 100.0)
mjd1 = ntmp2 * 1e-10 + ntmp1 * 1.0e-8
mjd1_gmrt_frac=mjd1
mjd1_gmrt_round = int(mjd1_gmrt) + mjd1_gmrt_frac#round(48.0 * (mjd1_gmrt - int(mjd1_gmrt))) / 48.0
#mjd1_gmrt_round_new = int(mjd1_gmrt_new) + mjd1_gmrt_frac_new
print(f"MJD_gmrt_round: {mjd1_gmrt_round}")

MJD_gmrt_round: 60297.6875


In [135]:
#Working Code that just recalculates mjd_start value from polyco 

import datetime
import math

# Given values
transit_lst_str = "08:36:09" #LST transit at afmjd
ra_str = "08h35m20.5554429s" #RA of Pulsar 
afmjd = 60297.90000185 #60268 #60297  # Example value for afmjd(MJD_start)
maxha = 6 #  maxha 
tspan = 60  #  tspan (min)

# Observatory coordinates  1656342.30    5797947.77 
obs_x =  1656342.30  # Replace with the actual x-coordinate
obs_y = 5797947.77 # Replace with the actual y-coordinate

# Convert transit LST to datetime
transit_lst_time = datetime.datetime.strptime(transit_lst_str, "%H:%M:%S").time()

# Convert RA to hours
ra_hours = (8 + (35/60) + (20.5554429/3600))

# Conversion of UT into local sidereal time
solsid = 1.002737909

#  LST transit in hours (taken from the NCRA transit time calculator)
hlst_gmrt = transit_lst_time.hour + transit_lst_time.minute/60 + transit_lst_time.second/3600 

# Calculate observatory longitude
alng = math.atan2(-obs_y, obs_x)

# Conversion of UT into local sidereal time
solsid = 1.002737909
hlst = 24.0 * ((solsid * afmjd + 0.154374 - alng / (2 * math.pi)) % 1.0)

 
rax = int(ra_hours) + int((ra_hours - int(ra_hours)) * 60.0 + 2) / 60.0

wait_gmrt = (rax - hlst ) / solsid #GMRT # Calculate wait : Local hour angle

# Add a day if wait is not in range #This is polyco criterion 
if wait_gmrt < -maxha:
    wait_gmrt += 24.0/solsid

# Calculate MJD1
mjd1_gmrt = afmjd + (wait_gmrt - (maxha)) / 24.0 + (tspan / (2.0 * 1440.0)) #- 0.008248211156023899
mjd1_gmrt_new = afmjd + ((rax - 24.0 * ((solsid * afmjd + 0.154374 - alng / (2 * math.pi)) % 1.0)) - maxha) / 24.0 + tspan / (2.0 * 1440.0)

mjd1_gmrt_frac = round(48.0 * (mjd1_gmrt - int(mjd1_gmrt))) / 48.0
mjd1_gmrt_frac_new = round(48.0 * (mjd1_gmrt - int(mjd1_gmrt))) / 48.0
ntmp1 = int(mjd1_gmrt_frac * 1.0e8)
ntmp2 = int((mjd1_gmrt_frac * 1.0e8 - ntmp1) * 100.0)
mjd1 = ntmp2 * 1e-10 + ntmp1 * 1.0e-8
mjd1_gmrt_frac=mjd1
mjd1_gmrt_round = int(mjd1_gmrt) + mjd1_gmrt_frac#round(48.0 * (mjd1_gmrt - int(mjd1_gmrt))) / 48.0
mjd1_gmrt_round_new = int(mjd1_gmrt_new) + mjd1_gmrt_frac_new
print(f"MJD_gmrt_round: {mjd1_gmrt_round}")

MJD_gmrt_round: 60297.6875


In [138]:
hlst

8.406038785062265

In [96]:
rax

8.616666666666667

In [97]:
wait_gmrt

-0.1883028282757363

In [100]:
#Test to check transit MJD
import datetime
import math
import numpy as np 
def calculate_wait(afmjd, maxha, tspan, obs_x, obs_y, transit_lst_str):
    #transit_lst_time = datetime.datetime.strptime(transit_lst_str, "%H:%M:%S").time()
    ra_hours = (8 + (35 / 60) + (20.5554429 / 3600))
    solsid = 1.002737909
    hlst = 24.0 * ((solsid * afmjd + 0.154374 - math.atan2(-obs_y, obs_x) / (2 * math.pi)) % 1.0)
    rax = int(ra_hours) + int((ra_hours - int(ra_hours)) * 60.0 + 2) / 60.0
    wait = (rax - hlst) / solsid
    if wait < -maxha:
        wait += 24.0 / solsid
    return wait

# Given values
#transit_lst_str = "08:36:09"
maxha = 6
tspan = 60
obs_x = 1656342.30
obs_y = 5797947.77

# Range for afmjd
afmjd_start = 60297.0
afmjd_end = 60300.0
afmjd_step = 0.001
mjd_range = np.arange(60296.0,60299.0,0.01)
afmjd_zero_wait = None

for afmjd in range(len(mjd_range)):
    wait_gmrt = calculate_wait(mjd_range[afmjd], maxha, tspan, obs_x, obs_y, transit_lst_str)
    #print(wait_gmrt)
    if abs(wait_gmrt) < 1e-1:
        print(wait_gmrt)
        print(mjd_range[afmjd])
        

0.03562757147547888
60296.910000000185
-0.029902833188359526
60297.91000000039
-0.09543323767805174
60298.91000000059


In [39]:
from astropy.time import Time
from astropy.coordinates import EarthLocation

# Observer's geocentric coordinates (replace with your values)
obs_x = 1656342.30 * u.m  # x-coordinate in meters
obs_y = 5797947.77 * u.m  # y-coordinate in meters
obs_z = 2073243.16 * u.m  # z-coordinate in meters

# Calculate Hour Angle at transit
transit_lst = ra_hours

# Normalize Hour Angle
#transit_lst = lst.hour - ha_at_transit
#transit_lst = transit_lst.wrap_at(24 * u.hourangle)  # Normalize to [0, 24) hours

print(f"Transit LST: {transit_lst}")


Transit LST: 8.589043178583333
